# 值得了解的python3新特性

### f-strings (3.6+)

In [1]:
name = 'Alice'
age = 8

# 之前的format方式
print('{} is {} years old.'.format(name, age))

# f-string方式
print(f'{name} is {age} years old.')

Alice is 8 years old.
Alice is 8 years old.


### Pathlib (3.4+)

In [2]:
from pathlib import Path

# 使用/号切换路径，使用resolve获取绝对路径
pwd = Path('.')
print((pwd/'golang').resolve())

/home/wangy/git/tech-blog/notes/golang


In [3]:
# 使用iterdir()遍历当前目录内容，仅遍历子目录
[item for item in pwd.iterdir() if item.is_dir()]

[PosixPath('deeplearning'),
 PosixPath('python-debug'),
 PosixPath('leetcode'),
 PosixPath('DLBible'),
 PosixPath('numpy'),
 PosixPath('java-design-patterns'),
 PosixPath('.ipynb_checkpoints'),
 PosixPath('SQLAlchemy'),
 PosixPath('golang')]

In [4]:
# 使用glob搜索所有markdown文件
[item for item in pwd.glob('**/*.md')]

[PosixPath('systemd_service_python.md'),
 PosixPath('deeplearning/keras-CNN-using-own-dataset.md'),
 PosixPath('java-design-patterns/README.md'),
 PosixPath('golang/50-common-gotchas-in-go.md')]

In [5]:
# 使用parts获得路径的所有组成部分
pwd.resolve().parts

('/', 'home', 'wangy', 'git', 'tech-blog', 'notes')

### 类型提示 (3.5+)

In [6]:
# 可以使用类型提示来说明变量的类型
def is_long_string(s: str) -> str:
    return len(s)>10

print(is_long_string('hello everyone.'))

True


### 迭代器 (3.4+)

In [7]:
# 使用Enum作为迭代器类的父类
from enum import Enum, auto
class Type(Enum):
    GET = auto()
    POST = auto()
    PUT = auto()
    DELETE = auto()
print([t for t in Type])

[<Type.GET: 1>, <Type.POST: 2>, <Type.PUT: 3>, <Type.DELETE: 4>]


### 内建的最近使用的缓存 LRU

In [8]:
# 使用lru_cache可以缓存一个函数的返回值，加速递归函数的调用
def normal_fib(n):
    if n==1 or n==2:
        return 1
    return normal_fib(n-1) + normal_fib(n-2)

from functools import lru_cache
@lru_cache(maxsize=100)
def lru_fib(n):
    if n==1 or n==2:
        return 1
    return lru_fib(n-1) + lru_fib(n-2)

In [9]:
# 不带缓存调用时间
%time normal_fib(40)

CPU times: user 24 s, sys: 259 µs, total: 24 s
Wall time: 24 s


102334155

In [10]:
# 带缓存调用时间
%time lru_fib(40)

CPU times: user 40 µs, sys: 0 ns, total: 40 µs
Wall time: 43.2 µs


102334155

### 扩展的迭代类型解包 (3.0+)

In [11]:
# 可以使用*符号对多个对应的集合（迭代）项目进行解包
head, *body, tail = range(10)
print(head, body, tail)

pyprogram, pyscript, *args = 'python main.py username groups 1 2 3'.split()
print(pyprogram, pyscript, args)

0 [1, 2, 3, 4, 5, 6, 7, 8] 9
python main.py ['username', 'groups', '1', '2', '3']


### Data classes (3.7+)

#### dataclass修饰器

dataclass一个python的修饰器，可以用来简便地创建创建python class，并自动生成一些特殊的类方法，如构造方法__init__，表示方法__repr__，对象相等比较方法__eq__等。示例：

```python
from dataclasses import dataclass

@dataclass
class MyClass():
    name: str
    age: int
    desc: str
```

会创建一个MyClass类，具有成员属性name, age和desc，（成员可以设置默认值，相当于`__init__`中的参数带有默认值），注意这里定义的成员必须指定类型。

```python
mc1 = MyClass('John', 40, 'John Doe') # 执行__init__构建对象
print(mc1) # 执行__repr__打印对象
mc2 = MyClass('John', 40, 'John Doe')
print(mc1==mc2) # 执行__eq__比较对象
```

输出：

    MyClass(name='John', age=20, desc='John Doe')
    True
    
dataclass修饰器的函数签名：

    @dataclasses.dataclass(*, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)

- order：是否创建`__lt__, __gt__, __le__, __ge__`方法
- hash：是否创建`__hash__`方法
- frozen：是否允许修改成员值，true的话对象将会成为不可变的

#### field类

field对象可以用来详细定义成员的设置，例如：

```python
from dataclasses import field

@dataclass
class MyClass():
    name: str
    age: int = None
    desc: str = field(default='No description', repr= False, init=False, compare=False)
```

field类的函数签名：

    dataclasses.field(*, default=MISSING, default_factory=MISSING, repr=True, hash=None, init=True, compare=True, metadata=None)

- default：默认值
- default_factory：默认值构造器，必须为一个无参的构造器，如`str, list, map`等
- repr：是否出现在`__repr__`方法中
- hash：是否作为哈希计算依据
- init：是否出现在`__init__`方法中
- compare：是否作为对象比较依据

[dataclass官方说明](https://docs.python.org/3/library/dataclasses.html)